<a href="https://colab.research.google.com/github/jorgeriva/MIAR-Algoritmos-Optimizacion/blob/main/Jorge_Rivadulla_Brey_Trabajo_Pr%C3%A1ctico_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Jorge Rivadulla Brey  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1d31gwNqCE2XSbdlhSStaOmVow03gheFp <br>
Problema:
Organizar los horarios de partidos de una jornada de La Liga<br>

Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la
audiencia.

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?

 El espacio de soluciones se encuentra definido por todas las posibles combinaciones de partidos y su posterior asignación en los horarios de la jornada. Es decir, se trata de todas las posibles jornadas que podemos implementar que cumplan con las restricciones estipuladas en el enunciado.


Cada posible solución dentro de este espacio puede representarse mediante una matriz, donde:

*   Las filas corresponden a los diferentes partidos que deben disputarse en la jornada
*   Las columnas representan los distintos horarios disponibles para la programación de los encuentros.


  En esta matriz, cada partido debe asignarse exclusivamente a un único horario, lo que implica que cada fila debe contener exactamente una selección válida dentro del conjunto de horarios disponibles. Este enfoque en forma de matriz facilita la visualización de una configuración particular de la jornada como la aplicación distintos algoritmos para obtener la solución óptima.

  Dado que el número de combinaciones es muy elevado el espacio de soluciones será de gran tamaño,

- ¿Cual es la función objetivo?

  El objetivo del problema es maximizar el valor de la jornada, de manera que los partidos que van a provocar una mayor audiencia se jueguen en los mejores horarios. Esto significa que la planificación de la jornada debe priorizar la asignación de encuentros de alto impacto en franjas horarias que garanticen una mayor visibilidad y, en consecuencia, una mayor audiencia global.

  Sabiendo esto, la función objetivo debe ser la maximización del sumatorio del producto del valor de los partidos de la jornada por sus horarios. Es decir, la función que nos dé como resultado la ordenación de los partidos que nos genera la jornada con una mayor audiencia posible, la cual se expresa de la siguiente forma:

$$
\max \sum_{i=1}^{N} V_i \cdot H_i
$$

* $V_i$ representa el valor asignado al partido $i$, determinado por el valor de la audiencia del partido por la correlacion si hay más partidos en ese horario.  

* $H_i$ es el coeficiente asociado al horario en el que se programa el partido $i$, reflejando el valor que tien cada uno de esos horarios.  

La sumatoria recorre todos los partidos programados en la jornada.



- ¿Como implemento las restricciones?

Contamos con tres restricciones, las cuales comentaremos aquí e implementaremos en el espacio para código que tenemos justo debajo.

1. Cada equipo solo puede jugar un partido. Con esto evitamos que un equipo aparezca en más de un espacio de la tabla de la jornada.
2. Solo se puede jugar en los horarios marcados. Para evitar que algún partido se salga de los espacios de la tabla.
3. Penalización por coincidencias: Si varios partidos coinciden en horario, su audiencia se reduce siguiendo las ponderaciones de la tabla que veremos más adelante.




In [ ]:
import numpy as np
import random
import pandas as pd

# Datos del problema
Partido_AA = 2
Partido_AB = 1.3
Partido_AC = 1
Partido_BB = 0.90
Partido_BC = 0.75
Partido_CC = 0.47

team_categories = {
    'Celta': 2, 'Real Madrid': 1, 'Valencia': 2, 'R. Sociedad': 1, 'Mallorca': 3,
    'Eibar': 3, 'Athletic': 2, 'Barcelona': 1, 'Leganes': 3, 'Osasuna': 3,
    'Villarreal': 2, 'Granada': 3, 'Alaves': 2, 'Levante': 2, 'Espanyol': 2,
    'Sevilla': 2, 'Betis': 2, 'Valladolid': 3, 'Atletico': 2, 'Getafe': 2
}

horarios = ["Sábado 20", "Domingo 20", "Domingo 18", "Sábado 18", "Domingo 16", "Sábado 16", "Sábado 12", "Domingo 12", "Viernes 20", "Lunes 20"]
coef_horario = {"Viernes 20": 0.4, "Sábado 12": 0.55, "Sábado 16": 0.7, "Sábado 18": 0.8, "Sábado 20": 1,
                "Domingo 12": 0.45, "Domingo 16": 0.75, "Domingo 18": 0.85, "Domingo 20": 1, "Lunes 20": 0.4}

# Coeficiente de penalización por coincidencia, esta es la tabla que muestra la penalización si
#dos equipos juegan su partido en la misma franja horaria
coef_coincidencia = {1: 1, 2: 0.75, 3:  0.65, 4: 0.4, 5: 0.30 , 6: 0.25, 7:0.22, 8: 0.20, 9 :0.20}

# 1. Representación del espacio de soluciones

# Generamos una jornada aleatoria
def generar_espacio_soluciones():
    equipos = list(team_categories.keys())
    random.shuffle(equipos)

    partidos_posibles = [(equipos[i], equipos[j]) for i in range(len(equipos)) for j in range(i + 1, len(equipos))]
    #Aplicamos las restricciones
    asignacion = aplicar_restricciones(partidos_posibles)

    # Mostrar la asignación de partidos con horarios y audiencia
    print("\nAsignación de partidos y horarios:")
    for partido, horario in asignacion.items():
        equipo1, equipo2 = partido
        nivel1, nivel2 = team_categories[equipo1], team_categories[equipo2]
        audiencia = calcular_audiencia_partido(equipo1, equipo2, horario, asignacion)
        print(f"{partido[0]} (Nivel {nivel1}) vs {partido[1]} (Nivel {nivel2}) -> {horario} | Aficionados: {audiencia:.2f} millones")

    return asignacion

# Función para calcular la audiencia de un partido específico, también lo podemos ver como una restricción
def calcular_audiencia_partido(equipo1, equipo2, horario, asignacion):
    nivel1, nivel2 = team_categories[equipo1], team_categories[equipo2]

    if nivel1 == 1 and nivel2 == 1:
        audiencia_base = Partido_AA
    elif (nivel1 == 1 and nivel2 == 2) or (nivel1 == 2 and nivel2 == 1):
        audiencia_base = Partido_AB
    elif (nivel1 == 1 and nivel2 == 3) or (nivel1 == 3 and nivel2 == 1):
        audiencia_base = Partido_AC
    elif nivel1 == 2 and nivel2 == 2:
        audiencia_base = Partido_BB
    elif (nivel1 == 2 and nivel2 == 3) or (nivel1 == 3 and nivel2 == 2):
        audiencia_base = Partido_BC
    elif nivel1 == 3 and nivel2 == 3:
        audiencia_base = Partido_CC

    # Penalizacion
    penalizacion_coincidencia = {}
    for h in asignacion.values():
        penalizacion_coincidencia[h] = penalizacion_coincidencia.get(h, 0) + 1
    coincidencias = penalizacion_coincidencia[horario]
    penalizacion = coef_coincidencia.get(coincidencias, 1)

    audiencia_con_penalizacion = audiencia_base * coef_horario[horario] * penalizacion

    return audiencia_con_penalizacion

# 3. Implementación de restricciones:
# - Asegurar que al menos haya un partido el viernes y otro el lunes.
# - Puede haber mas de un partido por fecha.

def aplicar_restricciones(partidos):
    asignacion = {}
    disponibles = list(horarios)
    usados = set()

    partidos_viernes = [partido for partido in partidos if partido[0] not in usados and partido[1] not in usados]
    partido_viernes = random.choice(partidos_viernes)
    asignacion[partido_viernes] = "Viernes 20"
    usados.add(partido_viernes[0])
    usados.add(partido_viernes[1])
    partidos.remove(partido_viernes)

    partidos_lunes = [partido for partido in partidos if partido[0] not in usados and partido[1] not in usados]
    if len(partidos_lunes) == 0:
        raise ValueError("No hay suficientes partidos disponibles para asignar a Lunes 20.")
    partido_lunes = random.choice(partidos_lunes)
    asignacion[partido_lunes] = "Lunes 20"
    usados.add(partido_lunes[0])
    usados.add(partido_lunes[1])
    partidos.remove(partido_lunes)

    for partido in partidos:
        partidos_disponibles = [p for p in partidos if p[0] not in usados and p[1] not in usados]

        if len(partidos_disponibles) == 0:
            return asignacion

        partido_disponible = random.choice(partidos_disponibles)

        if len(disponibles) == 0:
            return asignacion
        horario = random.choice(disponibles)

        asignacion[partido_disponible] = horario
        usados.add(partido_disponible[0])
        usados.add(partido_disponible[1])

    return asignacion

    def calcular_audiencia_total(asignacion):
      return sum(calcular_audiencia_partido(p[0], p[1], h, asignacion) for p, h in asignacion.items())


# Ejecución
asignacion = generar_espacio_soluciones()  # Generar la asignación de la jornada
audiencia_total = calcular_audiencia_total(asignacion)
print(f"\nAudiencia total de la jornada: {audiencia_total:.2f} millones de espectadores")



Asignación de partidos y horarios:
Villarreal (Nivel 2) vs Granada (Nivel 3) -> Viernes 20 | Aficionados: 0.30 millones
Atletico (Nivel 2) vs Levante (Nivel 2) -> Lunes 20 | Aficionados: 0.27 millones
Eibar (Nivel 3) vs Osasuna (Nivel 3) -> Domingo 20 | Aficionados: 0.47 millones
Sevilla (Nivel 2) vs Alaves (Nivel 2) -> Sábado 12 | Aficionados: 0.50 millones
Betis (Nivel 2) vs Athletic (Nivel 2) -> Sábado 20 | Aficionados: 0.59 millones
Celta (Nivel 2) vs Espanyol (Nivel 2) -> Sábado 20 | Aficionados: 0.59 millones
Real Madrid (Nivel 1) vs Mallorca (Nivel 3) -> Domingo 18 | Aficionados: 0.85 millones
Barcelona (Nivel 1) vs Valencia (Nivel 2) -> Lunes 20 | Aficionados: 0.39 millones
Valladolid (Nivel 3) vs Getafe (Nivel 2) -> Domingo 12 | Aficionados: 0.34 millones
Leganes (Nivel 3) vs R. Sociedad (Nivel 1) -> Sábado 20 | Aficionados: 0.65 millones


NameError: name 'calcular_audiencia_total' is not defined

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
import math
#Calculo de las distintas maneras que hay de combinar los diez partidos
equipos = 20
partidos = 10
producto_combinaciones = 1
for i in range(partidos):
    producto_combinaciones *= math.comb(equipos - 2*i, 2)
resultado = producto_combinaciones // math.factorial(partidos)



#Combinación para ordenar los partidos en una jornada.
#Para simplificarlo y diseñarlo nos hemos imaginado que las fechas de la jornada son
#cajas con numeros y los partidos pelotas. Calculamos las posibles combinaciones
#de meter las diez pelotas en las diez cajas, pero como dos pelotas tienen que ir si o si
#en las cajas uno y dos(Lunes y viernes) los calculos los hacemos sobre ocho pelotas
n = 8
k = 10
factor = resultado
combinaciones = math.comb(n + k - 1, k - 1)

# Multiplicamos por el numero de combinaciones de partidos por el numero de combinaciones de jornadas
resultado2 = combinaciones * factor
print(f"El número de combinaciones posibles para los 10 partidos es: {resultado}")
print(f"El número de jornadas posibles es : {resultado2}")






El número de combinaciones posibles para los 10 partidos es: 654729075
El número de jornadas posibles es : 15916463813250


El problema es una variación del problema de asignación donde el objetivo es maximizar la audiencia total. Tambien podemos ver que este problema es una mezcal entre el problema de asiganción de tareas y el problema de asigancion cuadrática, sabiendo esto podemos decir que es un problema NP-hard, por lo que encontrar una solución optima es dificil. Por lo tanto nos parece un buena solución el implementar un algoritmo voraz para resolverlo.

Este algoritmo esta formado por los siguientes apartados:
- Generación de partidos: O(N²)
- Ordenación: O(N² log N)
- Bucle principal: O(N²)
- Cálculo de audiencia total: O(1)

por lo cual su complejidad es O(N² log N)


Para contabilizar el espacio de soluciones debemos resolver un problema de combinatoria con la intención de ver cuantas posibles jornadas podrían llegar a producirse.

Lo primero sería aplicar C(20,2) con la idea de ver cuantas posibles combinaciones de partidos entre los equipo hay, cuyo resultado es 190 partidos posibles.

$$C(20, 2) = \frac{20!}{2!(20-2)!} = 190$$

Pero eso no sera todo, solo el principio, como lo que queremos es ver cuantas posibles combinaciones de diez partidos hay sin importar el orden lo que hay que jacer es un producto de combinaciones de la siguiente forma C(20,2),C(18,2),…,C(2,2) y, como no nos importa el orden, lo divideremos entre 10!.


$$\displaystyle \frac{C(20, 2) \cdot C(18, 2) \cdot \cdots \cdot C(2, 2)}{10!}$$




Con esto ya nos daría un numero extremadamente alto, y aun nos quedaría calcular las combinaciones de los partidos en la jornada, aun con la restricción de que al menos debe de haber un partido el lunes y el viernes. Para representar ese problema nos lo hemos imaginado como un problema de meter pelotas en cajas numeradas.


Primero calculamos las combinaciones para ordenar las pelotas (partidos) en las cajas (fechas) sin considerar restricciones:

$$C(n + k - 1, k - 1) = \binom{n+k-1}{k-1}$$

Donde $n = 8$ (el número de partidos restantes sin contar los dos que van si o si a lunes y viernes) y $k = 10$ (el número total de fechas), lo que nos da:

$$C(8 + 10 - 1, 10 - 1) = \binom{17}{9}$$

Luego, multiplicamos el resultado por el factor que representa el número de formas de ordenar los partidos, lo que da el total de combinaciones posibles para la jornada:

$$\text{Resultado total} = \binom{17}{9} \times \text{factor}$$


Como se ve el número de posibilidades crece exponencialmente tras realizar todos estos calculos, ya que los resultados tanto del número de posibilidades de jornadas como el del numero de posibles ordenes para una jornada deberían multiplicarse. Sabiendo esto acabaremos diciendo que el espacio de soluciones es exponencial.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [4]:
import numpy as np
import random
import pandas as pd

# Datos del problema
Partido_AA = 2
Partido_AB = 1.3
Partido_AC = 1
Partido_BB = 0.90
Partido_BC = 0.75
Partido_CC = 0.47

team_categories = {
    'Celta': 2, 'Real Madrid': 1, 'Valencia': 2, 'R. Sociedad': 1, 'Mallorca': 3,
    'Eibar': 3, 'Athletic': 2, 'Barcelona': 1, 'Leganes': 3, 'Osasuna': 3,
    'Villarreal': 2, 'Granada': 3, 'Alaves': 2, 'Levante': 2, 'Espanyol': 2,
    'Sevilla': 2, 'Betis': 2, 'Valladolid': 3, 'Atletico': 2, 'Getafe': 2
}

horarios = ["Sábado 20", "Domingo 20", "Domingo 18", "Sábado 18", "Domingo 16", "Sábado 16", "Sábado 12", "Domingo 12", "Viernes 20", "Lunes 20"]
coef_horario = {"Viernes 20": 0.4, "Sábado 12": 0.55, "Sábado 16": 0.7, "Sábado 18": 0.8, "Sábado 20": 1,
                "Domingo 12": 0.45, "Domingo 16": 0.75, "Domingo 18": 0.85, "Domingo 20": 1, "Lunes 20": 0.4}

# Coeficiente de penalización por coincidencia
coef_coincidencia = {1: 1, 2: 0.75, 3: 0.65, 4: 0.4, 5: 0.30, 6: 0.25, 7: 0.22, 8: 0.20, 9: 0.20}

# Función para calcular la audiencia de un partido específico
def calcular_audiencia_partido(equipo1, equipo2, horario, asignacion):
    nivel1, nivel2 = team_categories[equipo1], team_categories[equipo2]

    if nivel1 == 1 and nivel2 == 1:
        audiencia_base = Partido_AA
    elif (nivel1 == 1 and nivel2 == 2) or (nivel1 == 2 and nivel2 == 1):
        audiencia_base = Partido_AB
    elif (nivel1 == 1 and nivel2 == 3) or (nivel1 == 3 and nivel2 == 1):
        audiencia_base = Partido_AC
    elif nivel1 == 2 and nivel2 == 2:
        audiencia_base = Partido_BB
    elif (nivel1 == 2 and nivel2 == 3) or (nivel1 == 3 and nivel2 == 2):
        audiencia_base = Partido_BC
    elif nivel1 == 3 and nivel2 == 3:
        audiencia_base = Partido_CC

    # Penalización por coincidencias
    penalizacion_coincidencia = {}
    for h in asignacion.values():
        penalizacion_coincidencia[h] = penalizacion_coincidencia.get(h, 0) + 1
    coincidencias = penalizacion_coincidencia.get(horario, 1)
    penalizacion = coef_coincidencia.get(coincidencias, 1)

    return audiencia_base * coef_horario[horario] * penalizacion

# Función para calcular la audiencia total
def calcular_audiencia_total(asignacion):
    return sum(calcular_audiencia_partido(p[0], p[1], h, asignacion) for p, h in asignacion.items())

def obtener_maximo(partido):
    return max(Partido_AA, Partido_AB, Partido_AC, Partido_BB, Partido_BC, Partido_CC)

# Algoritmo voraz para maximizar la audiencia
def greedyalgorithm():
    equipos = list(team_categories.keys())
    partidos_posibles = [(equipos[i], equipos[j]) for i in range(len(equipos)) for j in range(i + 1, len(equipos))]

    partidos_posibles.sort(key=obtener_maximo, reverse=True)

    asignacion = {}
    disponibles = list(horarios)
    usados = set()

    for horario in horarios:
        mejor_partido = None
        mejor_audiencia = 0

        for partido in partidos_posibles:
            if partido[0] not in usados and partido[1] not in usados:
                audiencia = calcular_audiencia_partido(partido[0], partido[1], horario, asignacion)
                if audiencia > mejor_audiencia:
                    mejor_partido = partido
                    mejor_audiencia = audiencia

        if mejor_partido:
            asignacion[mejor_partido] = horario
            usados.add(mejor_partido[0])
            usados.add(mejor_partido[1])
            disponibles.remove(horario)

    return asignacion

# Ejecutar el algoritmo voraz
asignacion_optima = greedyalgorithm()
audiencia_total = calcular_audiencia_total(asignacion_optima)

# Mostrar los resultados
print("\nJornada óptima generada con algoritmo voraz:")
for partido, horario in asignacion_optima.items():
    print(f"{partido[0]} vs {partido[1]} -> {horario}")
print(f"\nAudiencia total de la jornada: {audiencia_total:.2f} millones de espectadores")



Jornada óptima generada con algoritmo voraz:
Real Madrid vs R. Sociedad -> Sábado 20
Celta vs Barcelona -> Domingo 20
Valencia vs Athletic -> Domingo 18
Villarreal vs Alaves -> Sábado 18
Levante vs Espanyol -> Domingo 16
Sevilla vs Betis -> Sábado 16
Atletico vs Getafe -> Sábado 12
Mallorca vs Eibar -> Domingo 12
Leganes vs Osasuna -> Viernes 20
Granada vs Valladolid -> Lunes 20

Audiencia total de la jornada: 7.17 millones de espectadores


Para resolver este problema, se tomó la decisión de utilizar algoritmos voraces, ya que se vio esta como una buena solución para el problema, tal y como se comentó en el apartado anterior. El razonamiento al que llegamos para la elección de este tipo de algoritmos como herramienta para la solución del problema fue gracias, en parte, a su definición:

El algoritmo voraz va tomando decisiones paso a paso; de esta manera, puede situar en los horarios con mayor número de audiencia los mejores partidos, algo muy útil para la resolución de este problema. Además, este tipo de algoritmo es muy sencillo de implementar y suele ser muy eficiente en tiempos de ejecución, lo que facilita la resolución del problema en comparación con otros algoritmos a los cuales, de tener que añadir las restricciones que este tiene, se nos habría aumentado mucho la complejidad. También, como con las restricciones, otra característica del problema que nos indica que pueden ser útiles los algoritmos voraces es el hecho de que cuenta con un espacio de datos pequeño, ya que el número de partidos que forma la jornada es únicamente diez.

Tal vez la solución se podría llegar a optimizar más si se usara un algoritmo híbrido mezclando, por ejemplo, técnicas voraces con una lista tabú para que vaya explorando posibles soluciones cercanas que mejorasen el resultado obtenido. Este algoritmo híbrido seguramente nos produciría una solución más eficiente que el algoritmo que hemos implementado, pero sus tiempos de ejecución serían mucho mayores, además de que su complejidad computacional y su complejidad de desarrollo aumentarían exponencialmente, por lo que al final hemos decidido quedarnos con esta implementación.